In [16]:
import pandas as pd
import numpy as np
# import tensorflow as tf
from packages import wrappers, models, ta
from sklearn.linear_model import LinearRegression

In [17]:
btcusd_df = wrappers.get_df_from_cryptowatch(pair='ethusd')

In [18]:
end_date = '2021-08-15'
num_day = 365

In [19]:
btcusd_df = models.add_TA(btcusd_df)
prepped_df = models.prepare_for_model(btcusd_df, end_date, num_day)
prepped_df

,Close,Volume,ATR_14,OBV,RSI_14,Target
Date,,,,,,
2020-08-14,432.39,2.006905e+07,25.852238,2.023126e+06,72.875351,433.67
2020-08-15,433.67,2.717136e+07,25.792078,2.087150e+06,73.094132,431.35
2020-08-16,431.35,3.055223e+07,25.873358,2.016556e+06,71.961191,422.62
2020-08-17,422.62,3.295368e+07,25.603118,1.938928e+06,67.708362,407.57
2020-08-18,407.57,3.725911e+07,26.143610,1.847700e+06,61.013904,416.20
...,...,...,...,...,...,...
2021-08-10,3164.50,1.602499e+08,195.612514,5.222545e+06,73.844800,3048.80
2021-08-11,3048.80,1.848556e+08,199.969477,5.162558e+06,67.197716,3324.02
2021-08-12,3324.02,1.681553e+08,206.524514,5.214775e+06,73.344286,3266.38


In [20]:
btcusd_df[btcusd_df.index <= pd.to_datetime(end_date)]

,Close,Volume,ATR_14,OBV,RSI_14,Target
Date,,,,,,
2015-08-07,1.19999,0.000000e+00,NaN,1.942888e+03,NaN,0.99000
2015-08-10,0.99000,0.000000e+00,NaN,-7.291681e+03,NaN,1.28800
2015-08-11,1.28800,0.000000e+00,NaN,-5.555566e+03,NaN,1.88480
2015-08-12,1.88480,0.000000e+00,NaN,-2.740115e+03,NaN,1.79489
2015-08-13,1.79489,0.000000e+00,NaN,-8.515924e+03,NaN,1.79000
...,...,...,...,...,...,...
2021-08-10,3164.50000,1.602499e+08,195.612514,5.222545e+06,73.844800,3048.80000
2021-08-11,3048.80000,1.848556e+08,199.969477,5.162558e+06,67.197716,3324.02000
2021-08-12,3324.02000,1.681553e+08,206.524514,5.214775e+06,73.344286,3266.38000


In [21]:
prepped_df[['Close', 'Volume', 'ATR_14', 'OBV', 'RSI_14']][:-1]

,Close,Volume,ATR_14,OBV,RSI_14
Date,,,,,
2020-08-14,432.39,2.006905e+07,25.852238,2.023126e+06,72.875351
2020-08-15,433.67,2.717136e+07,25.792078,2.087150e+06,73.094132
2020-08-16,431.35,3.055223e+07,25.873358,2.016556e+06,71.961191
2020-08-17,422.62,3.295368e+07,25.603118,1.938928e+06,67.708362
2020-08-18,407.57,3.725911e+07,26.143610,1.847700e+06,61.013904
...,...,...,...,...,...
2021-08-09,3141.69,2.141756e+08,198.881938,5.172652e+06,73.362432
2021-08-10,3164.50,1.602499e+08,195.612514,5.222545e+06,73.844800
2021-08-11,3048.80,1.848556e+08,199.969477,5.162558e+06,67.197716


In [22]:
prepped_df[['Target']]

,Target
Date,
2020-08-14,433.67
2020-08-15,431.35
2020-08-16,422.62
2020-08-17,407.57
2020-08-18,416.20
...,...
2021-08-10,3048.80
2021-08-11,3324.02
2021-08-12,3266.38


In [23]:
prepped_df[['Close', 'Volume', 'ATR_14', 'OBV']].iloc[[-1]]

,Close,Volume,ATR_14,OBV
Date,,,,
2021-08-14,3263.58,3.318602e+06,188.772668,5.184847e+06


In [24]:
lr = LinearRegression().fit(prepped_df[['Close', 'Volume', 'ATR_14', 'OBV', 'RSI_14']][:-1], prepped_df[['Target']][:-1])
prediction = lr.predict(prepped_df[['Close', 'Volume', 'ATR_14', 'OBV', 'RSI_14']].iloc[[-1]])
prediction

array([[3219.05906802]])

In [25]:
# models.generate_model('btcusd', prepped_df, '2020-01-01', 200, force_overwrite=True)

In [26]:
# pd.read_pickle('./models/btcusd_2020-01-01-linear-200/btcusd.pkl')

In [27]:
# lr = models.load_model('btcusd', '2020-01-01', 200)